In [85]:
using Random
using JSON3

In [86]:
function get_weights(x, max_interval)
    intervals = rand(0:max_interval, x-1)
    push!(intervals,0)
    push!(intervals,max_interval)
    intervals = sort(intervals)
    return diff(intervals)
end

function get_perm(n)
    P = Int16.(zeros(n,n))
    perm = randperm(n)
    for i=1:length(perm)
        P[perm[i], i] = 1
    end
    return P, perm
end

get_perm (generic function with 1 method)

In [87]:
n = 16
num_permutations = n*n

dataset_size = 100
dataset = Dict([])
max_interval = 10^(length(string(n*n)) + 2)

for i in range(0,dataset_size)
  
  X = zeros(n,n)
  permutations = zeros(n,num_permutations)
  weights = get_weights(num_permutations, max_interval)

  # generate doubly stochastic matrix
  for j=1:num_permutations
    P, perm = get_perm(n)
    X = X .+ weights[j]*P
    permutations[:,j] = perm
  end
  # save doubly stochastic matrix, weights, and permutations
  entry = Dict([("n",n),
    ("scale", max_interval), 
    ("scaled_doubly_stochastic_matrix", X),
    ("permutations", Int16.(permutations)), 
    ("weights", weights)
    ])
  dataset[i] = entry

end

file_name = "qbench/qbench_" * string(n) * "_dense.json"

open(file_name,"w") do io
  JSON3.pretty(io, dataset)
end

### Recover DSM from permutations and weights

In [88]:
using JSON3
n = 3
data = JSON3.read("data/dataset_" * string(n) * "_sparse.json")
D = data["1"].doubly_stochastic_matrix
D = reshape(D,n,n)

num_perm = n

M = Int16.(zeros(n*n,num_perm))
perm = Int16.(reshape(data["1"].permutations, n, num_perm))
for i=1:num_perm
    for j=1:n
        M[(j-1)*n + perm[j,i],i] = 1
    end
end

ArgumentError: ArgumentError: invalid JSON at byte position 1 while parsing type Any: InvalidChar
data/dataset_3_sparse.json


In [89]:
reshape(M*Float64.(data["1"].weights), n, n)

UndefVarError: UndefVarError: `M` not defined

In [90]:
reshape(data["1"].doubly_stochastic_matrix, n, n)

UndefVarError: UndefVarError: `data` not defined